<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/main/20221119_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%B8%EB%93%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys
from datetime import datetime, timedelta
from tqdm.autonotebook import tqdm
import csv
import re
import gzip
import shutil

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [ ]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

Mounted at /content/gdrive/


['CHANGELOG.txt',
 'LICENSE.txt',
 'SHA256SUMS.txt',
 'index.html',
 'icu',
 'hosp',
 'drop_2_chartevents.csv']

In [ ]:
admission = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv', low_memory=False, dtype='str', keep_default_na=False)
patients = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/patients.csv', low_memory=False, dtype='str', keep_default_na=False)
diagnoses_icd = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/diagnoses_icd.csv', low_memory=False, dtype='str', keep_default_na=False)
icustays = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/icu/icustays.csv', low_memory=False, dtype='str', keep_default_na=False)
sepsis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/sepsis3.csv', low_memory=False, dtype='str', keep_default_na=False)

In [ ]:
admission = pd.merge(left = admission, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
patients = pd.merge(left = patients, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
diagnoses_icd = pd.merge(left = diagnoses_icd, right = sepsis[['subject_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
sepsis = pd.merge(left = sepsis, right = icustays[['hadm_id','stay_id']], how='left', left_on='stay_id', right_on='stay_id')

In [ ]:
extra_features = pd.read_csv('/content/gdrive/MyDrive/mimiciv preprocessing_js/extra_features2.csv', low_memory=False, keep_default_na=False)
sepsis2 = pd.merge(left = sepsis, right = extra_features['hadm_id'].astype(str), how='inner', left_on='hadm_id', right_on='hadm_id')

In [ ]:
id_cepha = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/temp/id_cepha.csv', low_memory=False, dtype='str', keep_default_na=False)
id_glyco = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/temp/id_glyco.csv', low_memory=False, dtype='str', keep_default_na=False)
id_peni = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/temp/id_peni.csv', low_memory=False, dtype='str', keep_default_na=False)
id_fluoro = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/temp/id_fluoro.csv', low_memory=False, dtype='str', keep_default_na=False)
id_macro = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/temp/id_macro.csv', low_memory=False, dtype='str', keep_default_na=False)
id_cepha['LABEL'] = "cepha"
id_glyco['LABEL'] = "glyco"
id_peni['LABEL'] = "peni"
id_fluoro['LABEL'] = "fluoro"
id_macro['LABEL'] = "macro"

id = pd.concat([id_cepha,id_glyco,id_peni,id_fluoro,id_macro])

id.groupby(['LABEL']).count()

In [ ]:
Total_HADMIDS = set(sepsis2['hadm_id'])
len(Total_HADMIDS)
# Total_HADMIDS = set(sepsis['hadm_id'])
# len(Total_HADMIDS)
#32812

32365

In [ ]:
# feature_keys = ['FiO2', 'HR', 'PaO2', 'SBP', 'Temperature','Creatinine','Glucose', 'Hb', 'Lactate', 'Plt','Sodium' ,'WBC']
# feature_keys = ['FiO2', 'HR', 'PaO2', 'SBP', 'Temperature','Creatinine','Glucose', 'Hb', 'Lactate', 'Plt','Sodium' ,'WBC', 'DBP', 'MAP', 'RESP', 'Chloride', 'Hct', 'PCO2', 'Potassium', 'PH', 'Anion Gap', 'Bilirubin']
feature_keys = ['FiO2', 'HR', 'PaO2', 'SBP', 'Temperature','Creatinine','Glucose', 'Hb', 'Lactate', 'Plt','Sodium' ,'WBC', 'DBP', 'MAP', 'RESP', 'Chloride', 'Hct', 'PCO2', 'Potassium', 'PH', 'Anion Gap', 'Bilirubin']
print(len(feature_keys))

22


In [ ]:
EXTRACT_FEATURE_CODES_VS = dict({
    '226754': 'FiO2', '223835': 'FiO2', '227010': 'FiO2', '229280':'FiO2', '229841': 'FiO2',
    '220045': 'HR', '220046': 'HR', '226764': 'HR', '227018': 'HR',
    '220224': 'PaO2', '227039': 'PaO2', '227516': 'PaO2',
    '220050': 'SBP' ,'220179': 'SBP', '224167': 'SBP', '227243': 'SBP', '228152': 'SBP',
    '223762': 'Temperature', '226329': 'Temperature', '227632': 'Temperature', '227634': 'Temperature',
    '223761': 'TEMP_F',
    '225310': 'DBP', '220051': 'DBP', '220180': 'DBP',
    '225312': 'MAP', '220052': 'MAP', '220181': 'MAP',
    '220210': 'RESP', '224690': 'RESP'
    })
EXTRACT_FEATURE_CODES_LAB = dict({
    '220615': 'Creatinine', '229761': 'Creatinine',
    '52024': 'Creatinine', '50912': 'Creatinine', '52546': 'Creatinine', '51977': 'Creatinine',  '51081': 'Creatinine',
    '220621': 'Glucose', '225664': 'Glucose', '226537': 'Glucose',
    '50809': 'Glucose', '52027': 'Glucose', '50931': 'Glucose', '52569': 'Glucose', '51981': 'Glucose', '51478': 'Glucose',
    '220228': 'Hb',
    '50811': 'Hb', '50855': 'Hb', '51640': 'Hb', '51645': 'Hb', '51222': 'Hb', '52157': 'Hb',
    '225668': 'Lactate',
    '50813': 'Lactate', '52442': 'Lactate',
    '227457': 'Plt',
    '51704': 'Plt', '51265': 'Plt',
    '220645': 'Sodium', '226534': 'Sodium',
    '50824': 'Sodium', '52455': 'Sodium', '50983': 'Sodium', '52623': 'Sodium',
    '220546': 'WBC',
    '51755': 'WBC', '51756': 'WBC', '51301': 'WBC',
    '50805': 'Chloride', '50902': 'Chloride',
    '50810': 'Hct',
    '50810': 'PCO2',
    '50822': 'Potassium', '50971': 'Potassium',
    '50820': 'PH',
    '50868': 'Anion Gap',
    '50885': 'Bilirubin'
    }) 
EXTRACT_FEATURE_CODES_NORMAL = dict({
        'FiO2': [10, 100],
        'HR': [30, 200],
        'PaO2': [50, 200],
        'SBP': [40, 250],
        'Temperature': [30, 45],
        'Creatinine': [0.2, 100],
        'Glucose': [30, 1000],
        'Hb': [5, 20],
        'Lactate': [0.5, 2.5],
        'Plt': [10, 500],
        'Sodium': [90, 170],
        'WBC': [1, 30],
        'DBP':[30,200],
        'MAP':[33,216],
        'RESP':[0,35],
        'Chloride':[70,130],
        'Hct':[10,50],
        'PCO2':[5,100],
        'Potassium':[1.0,10.0],
        'PH':[5.5,8.0],
        'Anion Gap':[5,50],
        'Bilirubin':[0.1,10]
})
NORMAL_VALUE = dict({
        'FiO2': 40,
        'HR': 75, #(50+100)/2
        'PaO2': 87.5, #(75+100)/2
        'SBP': 115, #(90+140)/2 = 115
        'Temperature': 36.65, #(36.1+37.2)/2
        'Creatinine': 1.0, #(0.7+1.0)/2
        'Glucose': 100, #(60+140)/2
        'Hb': 15.0, #(13.8+17.2)/2
        'Lactate': 1.35, #(0.5+2.2)/2 = 1.35
        'Plt': 270, #(140+400)/2
        'Sodium': 140, #(135+145)/2
        'WBC': 7.5, #(5+10)/2
        'DBP': 90, #(80+100)/2 
        'MAP': 98, # (115 + 2*90) / 
        'RESP': 13, 
        'Chloride': 102, #(96+108) / 2
        'Hct': 34.5, #(34+45)/2
        'PCO2': 40, #(35+45)/2
        'Potassium': 4.5, #(3.5+5.4)/2
        'PH': 7.4, #(7.35 + 7.45) / 2
        'Anion Gap': 14, #8+20/2
        'Bilirubin': 0.8 #0+1.5/2
})

# EXTRACT_FEATURE_CODES_anti = ('225798','225837','225838','225840','225842','225843','225844','225845','225847','225848','225850','225851','225853','225855','225857','225859','225860','225862','225863','225865','225866','225868','225869','225871','225873','225875','225876','225877','225879','225881','225882','225883','225884','225885','225886','225888','225889','225890','225892','225893','225895','225896','225897','225898','225899','225900','225902','225903','225905','227691','228003','229059','229061','229064','229587')
# EXTRACT_FEATURE_CODES_ANTI_2 = dict({
#    '225850': 'Cephalosporin', '225851': 'Cephalosporin', '225853': 'Cephalosporin', '225855': 'Cephalosporin', '229587':'Cephalosporin',
#    '225798': 'Glycopetide',
#    '225842': 'Penicillin', '225843': 'Penicillin', '225888': 'Penicillin', '225889': 'Penicillin', '225890': 'Penicillin', '225892': 'Penicillin', '225893': 'Penicillin',
#    '225859': 'Quninolone', '225859': 'Quninolone', '225879': 'Quninolone', '225886': 'Quninolone',
#    '225845': 'Macrolide', '225865': 'Macrolide', '225866': 'Macrolide'
# })

In [ ]:
## Define some util functions
NOT_CONVERTED = 'NOT_CONVERTED'
## 문자로 기록된 날짜 값들을 파이썬의 datetime 데이터로 바꿔주는 함수 작성
def str2datetime(s):
    def _convert(_s, _dformat):
        try:
            converted_dt = datetime.strptime(_s, _dformat)
        except Exception:
            return NOT_CONVERTED

        return converted_dt

    if isinstance(s, datetime):
        return s

    dformats = [
        '%Y-%m-%d %p %I:%M:%S',
        '%Y-%m-%d %H:%M:%S.%f',
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d %H:%M',
        '%y-%m-%d %H:%M',
        '%Y-%m-%d',
        '%Y%m%d',
        '%Y-%m-%d +%H:%M',
        '%Y%m%d %H%M',
        '%Y%m%d%H%M%S',
        ]

    s = s.split('.')[0]
    s = s.replace('오전', 'am').replace('오후', 'pm')
    for dformat in dformats:
        dt = _convert(s, dformat)
        if dt != NOT_CONVERTED:
            return dt

    return NOT_CONVERTED

In [ ]:
def extract_features():
  features_hadm_id = dict()
  chartevents = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_unzip/icu/chartevents.csv', 'r', encoding='utf-8'))
  labevents = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_unzip/hosp/labevents.csv', 'r', encoding='utf-8'))
  for row in tqdm(chartevents, desc='Extract_VS', total=329822285, ncols=700):
    if row['hadm_id'] in Total_HADMIDS:
      hadm_id = row['hadm_id']
      features_hadm_id[hadm_id] = features_hadm_id.get(hadm_id, dict()) #get 없으면 2번째로 return

      item_id = row['itemid']
      if item_id not in EXTRACT_FEATURE_CODES_VS:
        continue
      item_name = EXTRACT_FEATURE_CODES_VS[item_id]
      try:
        #정규식을 이용해서 기록된 값에서 숫자 형태의 값만 가져오기
        #소수점 둘째자리 까지만 값을 사용하기
        item_value = round(float(re.findall("\d+\.\d+|\d+", row['valuenum'])[0]), 2)
      except Exception:
        continue
      
      item_time = str2datetime(row['charttime'])
      if item_time == NOT_CONVERTED:
        continue
      #GCP total score도 시간대에 총합으로 사용함
      if item_name == 'TEMP_F':
        item_value = round((item_value - 32.0) / 1.8, 2)
        item_name = 'TEMP'

    #   if item_value > FEATURE_NORMAL_RANGE[item_name][1]:
    #     item_value = FEATURE_NORMAL_RANGE[item_name][1]
    #   elif item_value < FEATURE_NORMAL_RANGE[item_name][0]:
    #     item_value = FEATURE_NORMAL_RANGE[item_name][0]

      features_hadm_id[hadm_id][item_name] = features_hadm_id[hadm_id].get(item_name, list()) #같은 환자의 여러개 feature가 있으면 list로 반환하도록
      features_hadm_id[hadm_id][item_name].append((item_time, item_value))
      
    else:
      continue

  for row in tqdm(labevents, desc='Extract_LAB', total=124342638, ncols=700):
    if row['hadm_id'] in Total_HADMIDS:
      hadm_id = row['hadm_id']
      features_hadm_id[hadm_id] = features_hadm_id.get(hadm_id, dict())

      item_id = row['itemid']
      if item_id not in EXTRACT_FEATURE_CODES_LAB:
        continue
      item_name = EXTRACT_FEATURE_CODES_LAB[item_id]
      try:
        #정규식을 이용해서 기록된 값에서 숫자 형태의 값만 가져오기
        #소수점 둘째자리 까지만 값을 사용하기
        item_value = round(float(re.findall("\d+\.\d+|\d+", row['valuenum'])[0]), 2)
      except Exception:
        continue
      
      item_time = str2datetime(row['charttime'])
      if item_time == NOT_CONVERTED:
        continue

      #if item_value > FEATURE_NORMAL_RANGE[item_name][1]:
      #  item_value = FEATURE_NORMAL_RANGE[item_name][1]
      #elif item_value < FEATURE_NORMAL_RANGE[item_name][0]:
      #  item_value = FEATURE_NORMAL_RANGE[item_name][0]
        
      features_hadm_id[hadm_id][item_name] = features_hadm_id[hadm_id].get(item_name, list())
      features_hadm_id[hadm_id][item_name].append((item_time, item_value))
      
    else:
      continue
      
  return features_hadm_id

In [ ]:
# extracted_features = extract_features()

Extract_VS:   0%|                                                                                             …

Extract_LAB:   0%|                                                                                            …

In [ ]:
import pickle
import gzip

In [ ]:
with gzip.open('/content/gdrive/MyDrive/sepsis_model_weight/extracted_features_1112.pickle','rb') as f:
    extracted_features = pickle.load(f)

extra_features.set_index('hadm_id', drop=True, append=False, inplace=True, verify_integrity=False)
extra_features=extra_features[['weight','age']]

In [ ]:
extra_features

,weight,age
hadm_id,,
27370071,89.000000,53
27939019,89.000000,44
27658791,48.700000,91
21077260,62.100000,43
21187549,62.100000,43
...,...,...
21874598,67.800000,62
24735133,67.800000,61
27024433,63.400000,23


In [ ]:
# with gzip.open('/content/gdrive/MyDrive/sepsis_model_weight/extracted_features.pickle','rb') as f:
#     extracted_features = pickle.load(f)

In [ ]:
# 사망 레이블 하기
death_labels = dict()
admissions = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_admissions.csv', 'r', encoding='utf-8'))
for row in tqdm(admissions, desc='Get Death Label'):
  hadm_id = row['hadm_id']
  death_yn = int(row['hospital_expire_flag'])
  if death_yn:
    event_time = str2datetime(row['deathtime'])
  else:
    event_time = str2datetime(row['dischtime'])
  death_labels[hadm_id] = dict({'DEATH': death_yn, 'EVENT_TIME': event_time})

Get Death Label: 0it [00:00, ?it/s]

In [ ]:
# 재실 기간 레이블 하기
los_labels = dict()
admissions = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_admissions.csv', 'r', encoding='utf-8'))
for row in tqdm(admissions, desc='Get loss Label'):
  hadm_id = row['hadm_id']
  admin_time = str2datetime(row['admittime'])
  disc_time = str2datetime(row['dischtime'])
  los = (disc_time - admin_time).total_seconds() / (60*60)
  los_labels[hadm_id] = int(los)

Get loss Label: 0it [00:00, ?it/s]

In [ ]:
## 환자의 입실 퇴실 시간 정보 가져오기
Extracted_HADMIDS = list(extracted_features.keys())
admin_disc_time = dict()

admissions = csv.DictReader(open('/content/gdrive/MyDrive/mimiciv_der/sepsis3_admissions.csv', 'r', encoding='utf-8'))
for row in admissions:
  hadm_id = row['hadm_id']
  admin_time = str2datetime(row['admittime'])
  disc_time = str2datetime(row['dischtime'])

  if (admin_time != NOT_CONVERTED) and (disc_time != NOT_CONVERTED):
    admin_disc_time[hadm_id] = dict({'ADMIN': admin_time, 'DISC': disc_time})

In [ ]:
## 입실 시점으로부터 1시간 간격으로 데이터를 읽어오는 함수 정의
def quantize_features(feature_list, admin_time, disc_time, missing_feature=False):
  total_hours = int((disc_time - admin_time).total_seconds() / (60*60))
  #입실 이후 최대 24시간 동안의 데이터만 가져오기
  if total_hours > 24:
    total_hours = 24
  
  quantized_features = list()
  if missing_feature:
    for i in range(total_hours):
      quantized_time = admin_time + timedelta(hours=i)
      quantized_value = np.nan #지금은 dic 형태라, pandas 쓰면 NAN이랑 동일, carry forward 도 가능
      quantized_features.append((quantized_time, quantized_value))
  else:
    feature_list = list(sorted(feature_list, key=lambda x: x[0]))
    for i in range(total_hours):
      quantized_time = admin_time + timedelta(hours=i)
      quantized_time_start = quantized_time - timedelta(hours=1)
      quantized_values = list(filter(lambda x: quantized_time_start < x[0] <= quantized_time, feature_list)) #filter함수 True만 필터
      if len(quantized_values) > 0:
        quantized_value = quantized_values[-1][1]
      else:
        quantized_value = np.nan 

      quantized_features.append((quantized_time, quantized_value))
  
  return quantized_features

In [ ]:
## 전체 데이터에 대해서 데이터 변환하기
quantized_features = dict()
for hadm_id in tqdm(Extracted_HADMIDS, desc='Doing Quantization'):
    admin_time = admin_disc_time[hadm_id]['ADMIN']
    disc_time = admin_disc_time[hadm_id]['DISC']
    if (disc_time - admin_time).total_seconds() / (60*60) < 1.0:
      continue
    quantized_features[hadm_id] = dict()
    for kk in feature_keys:
      if kk in extracted_features[hadm_id]:
        feature_list = extracted_features[hadm_id][kk]
        quantized_features[hadm_id][kk] = quantize_features(feature_list, admin_time, disc_time)
      else:
        quantized_features[hadm_id][kk] = quantize_features(None, admin_time, disc_time, missing_feature=True)

Doing Quantization:   0%|          | 0/32365 [00:00<?, ?it/s]

In [ ]:
def carry_forward(feature_name, feature_list):
  imputed_features = list()
  for i, (feature_t, feature_v) in enumerate(feature_list):
    if np.isnan(feature_v):
      last_observations = list(filter(lambda x: (x[0] <= feature_t) and (not np.isnan(x[1])), feature_list))
      if len(last_observations) > 0:
        imputed_features.append((feature_t, last_observations[-1][1]))
      else:
        imputed_features.append((feature_t, NORMAL_VALUE[feature_name]))
    else:
      imputed_features.append((feature_t, feature_v))

  return imputed_features

In [ ]:
Quantized_HADMIDS = list(quantized_features.keys())
carry_forwarded_features = dict()
for hadm_id in tqdm(Quantized_HADMIDS, desc='Carry-Forward'):
  carry_forwarded_features[hadm_id] = dict()

  for kk in feature_keys:
    feature_list = quantized_features[hadm_id][kk]
    carry_forwarded_features[hadm_id][kk] = carry_forward(kk, feature_list)

Carry-Forward:   0%|          | 0/32340 [00:00<?, ?it/s]

In [ ]:
#Min-Max Normalization
min_max_normalized = dict()
for hadm_id in tqdm(Quantized_HADMIDS, desc='Min-Max Normalization', ncols=700):
  min_max_normalized[hadm_id] = dict()
  for kk in feature_keys:
    features = np.array(carry_forwarded_features[hadm_id][kk])

    min = EXTRACT_FEATURE_CODES_NORMAL[kk][0]
    max = EXTRACT_FEATURE_CODES_NORMAL[kk][1]

    features[:, 1] = (features[:, 1] - min) / (max - min)
    min_max_normalized[hadm_id][kk] = features

Min-Max Normalization:   0%|                                                                                  …

In [ ]:
total_death_included_hadm_ids = list(death_labels.keys())
total_los_included_hadm_ids = list(los_labels.keys())

In [ ]:
#@title ##사망용 데이터 만들기

death_dataset = dict({
    'HADM_IDS': list(),
    'LABEL': list(),
    'DATA': list(),
    'EXTRA' : list(),
})

for hadm_id in tqdm(total_death_included_hadm_ids, desc='Make Death Dataset', ncols=700):
  if hadm_id not in min_max_normalized:
    continue
  if len(min_max_normalized[hadm_id][feature_keys[0]]) >= 24:
    time_len = 24
  else:
    time_len = len(min_max_normalized[hadm_id][feature_keys[0]])
  
  try:
    data = np.array(min_max_normalized[hadm_id][feature_keys[0]])[:time_len, 1].reshape(time_len, 1)
  except:
    continue

  for kk in feature_keys[1:]:
    data = np.concatenate((data, np.array(min_max_normalized[hadm_id][kk])[:time_len, 1].reshape(time_len, 1)), axis=1)
  
  death_dataset['HADM_IDS'].append(hadm_id)
  death_dataset['LABEL'].append(death_labels[hadm_id]['DEATH'])
  death_dataset['DATA'].append(data)

# 추가 feature 입력
extra_features=extra_features.reindex(pd.to_numeric(death_dataset['HADM_IDS']))
extra_features=np.array(extra_features)

for ef in tqdm(extra_features, desc='+ Extra_features', ncols=700):
  death_dataset['EXTRA'].append(ef)

Make Death Dataset:   0%|                                                                                     …

+ Extra_features:   0%|                                                                                       …

In [1]:
#@title ## 항생제 레이블 하기
anti_labels = dict()
for row in list(set(id['hadm_id'])):
    hadm_id = row
    anti_1 = 0
    anti_2 = 0
    anti_3 = 0
    anti_4 = 0
    anti_5 = 0
    temp = id[id['hadm_id']==row]
    temp = temp.groupby(['LABEL']).count()
    if 'cepha' in temp.iloc[:,0] : 
        anti_1 = 1
    if 'glyco' in temp.iloc[:,0] :
        anti_2 = 1
    if 'peni' in temp.iloc[:,0] :
        anti_3 = 1
    if 'fluoro' in temp.iloc[:,0] :
        anti_4 = 1
    if 'macro' in temp.iloc[:,0] :
        anti_5 = 1
    anti_labels[hadm_id] = dict({'cepha': anti_1, 'glyco':anti_2, 'peni':anti_3, 'fluoro':anti_4, 'macro':anti_5})

SyntaxError: ignored

In [ ]:
#@title ## 항생제용 데이터 만들기
anti_dataset = dict({
    'HADM_IDS': list(),
    'LABEL_1': list(),
    'LABEL_2': list(),
    'LABEL_3': list(),
    'LABEL_4': list(),
    'LABEL_5': list(),
    'DATA': list(),
    'EXTRA' : list()
})

for hadm_id in tqdm(total_death_included_hadm_ids, desc='Make anti Dataset', ncols=700):
  if hadm_id not in min_max_normalized:
    continue
  if len(min_max_normalized[hadm_id][feature_keys[0]]) >= 24:
    time_len = 24
  else:
    time_len = len(min_max_normalized[hadm_id][feature_keys[0]])
  
  try:
    data = np.array(min_max_normalized[hadm_id][feature_keys[0]])[:time_len, 1].reshape(time_len, 1)
  except:
    continue

  for kk in feature_keys[1:]:
    data = np.concatenate((data, np.array(min_max_normalized[hadm_id][kk])[:time_len, 1].reshape(time_len, 1)), axis=1)
  
  anti_dataset['HADM_IDS'].append(hadm_id)
  try :
      anti_dataset['LABEL_1'].append(anti_labels[hadm_id]['cepha'])
  except : 
      anti_dataset['LABEL_1'].append(0)
  try :
      anti_dataset['LABEL_2'].append(anti_labels[hadm_id]['glyco'])
  except : 
      anti_dataset['LABEL_2'].append(0)
  try :
      anti_dataset['LABEL_3'].append(anti_labels[hadm_id]['peni'])
  except : 
      anti_dataset['LABEL_3'].append(0)
  try :
      anti_dataset['LABEL_4'].append(anti_labels[hadm_id]['fluoro'])
  except : 
      anti_dataset['LABEL_4'].append(0)
  try :
      anti_dataset['LABEL_5'].append(anti_labels[hadm_id]['macro'])
  except : 
      anti_dataset['LABEL_5'].append(0)

  anti_dataset['DATA'].append(data)

# 추가 feature 입력
if type(extra_features) is np.ndarray : 
    pass
else :
    extra_features=extra_features.reindex(pd.to_numeric(anti_dataset['HADM_IDS']))
    extra_features=np.array(extra_features)

for ef in tqdm(extra_features, desc='+ Extra_features', ncols=700):
  anti_dataset['EXTRA'].append(ef)

print(len(anti_dataset['HADM_IDS']))

Make anti Dataset:   0%|                                                                                      …

+ Extra_features:   0%|                                                                                       …

32340
